In [ ]:
import pandas as pd
import folium


In [ ]:
df = pd.read_excel('/content/policee.xlsx')
df.head()

In [ ]:
df.shape

In [ ]:
df_inc = df.iloc[0:500,:]
df_inc.shape
df_inc.columns

Index(['Incident Datetime', 'Incident Date', 'Incident Time', 'Incident Year',
       'Incident Day of Week', 'Report Datetime', 'Row ID', 'Incident ID',
       'Incident Number', 'CAD Number', 'Report Type Code',
       'Report Type Description', 'Filed Online', 'Incident Code',
       'Incident Category', 'Incident Subcategory', 'Incident Description',
       'Resolution', 'Intersection', 'CNN', 'Police District',
       'Analysis Neighborhood', 'Supervisor District', 'Latitude', 'Longitude',
       'point', 'SF Find Neighborhoods', 'Current Police Districts',
       'Current Supervisor Districts', 'Analysis Neighborhoods',
       'HSOC Zones as of 2018-06-05', 'OWED Public Spaces',
       'Central Market/Tenderloin Boundary Polygon - Updated',
       'Parks Alliance CPSI (27+TL sites)', 'ESNCAG - Boundary File',
       'Areas of Vulnerability, 2016'],
      dtype='object')

SanFranciso Crime Scene/Incident Mapping
Get the lat and lon value of that city

In [ ]:
lat = 37.56
lon = -122.28
sanfrans_map = folium.Map(location=[lat,lon],zoom_start=8)
type(sanfrans_map)

folium.folium.Map

In [ ]:
lat_mean = df_inc['Latitude'].mean(skipna=True)
lon_mean = df_inc['Longitude'].mean(skipna=True)
df_inc['Latitude'].fillna(lat_mean,inplace=True)
df_inc['Longitude'].fillna(lon_mean,inplace=True)


In [ ]:
inc = folium.map.FeatureGroup()
for latt, lng in zip(df_inc.Latitude, df_inc.Longitude):
  inc.add_child(
      folium.CircleMarker(
          [latt,lng],
          radius = 5,
          color = 'red',
          fill = True,
          fill_color='blue',
          opacity=0.6
      )
  )

#add some more features on map

latitudes = list(df_inc.Latitude)
longitudes = list(df_inc.Longitude)
labels = list(df_inc['Incident Category'])

for lat, lng, labels in zip(latitudes,longitudes,labels):
  folium.Marker([lat,lng], popup=labels).add_to(sanfrans_map)



sanfrans_map.add_child(inc)

CLustering Approach

In [ ]:
from folium import plugins
sanfrans_map1 = folium.Map(location=[lat,lon],zoom_start=8)
incidents = plugins.MarkerCluster().add_to(sanfrans_map1)
for lat, lng, label in zip(df_inc.Latitude, df_inc.Longitude, df_inc['Incident Category']):
  folium.Marker(
      location=[lat,lng],
      icon = None,
      popup = label,

  ).add_to(incidents)

sanfrans_map1


Choropleth Map: thematic map used for shadding area.

In [ ]:
df_can = pd.read_excel('/content/canada.xlsx',sheet_name='Canada by Citizenship',
                    skipfooter =2, skiprows=range(20))
df_can.head()

In [ ]:
df_can.rename(columns={'OdName':'Country', 'AreaName':'Continent','RegName':'Region'}, inplace=True)


df_can.columns = list(map(str, df_can.columns))

df_can['Total'] = df_can.sum(axis=1)

years = list(map(str, range(1980, 2014)))
df_can.head()


In [ ]:
#load josn file and save it into variable
import json
f = open('world1.json')
world_geo = json.load(f)
x = pd.read_json('world1.json')


In [ ]:
world_geo

In [ ]:
world_map = folium.Map(location=[0,0],zoom_start=2,tiles='Mapbox Bright')
world_map.choropleth(
    geo_data = world_geo,
    data = df_can,
    columns =['Country','Total'],
    key_on='feature.properties.name',
    fill_color = 'YlGnBu',
    fill_opacity=0.6,
    
    line_opcaity=0.2,
    legend ='Canada',
    reset=True
)
folium.LayerControl().add_to(world_map)
world_map

/usr/local/lib/python3.6/dist-packages/folium/folium.py:426: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [ ]:
dat = pd.read_csv('master.csv')
dat.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


In [ ]:
dat.columns

Index(['country', 'year', 'sex', 'age', 'suicides_no', 'population',
       'suicides/100k pop', 'country-year', 'HDI for year',
       ' gdp_for_year ($) ', 'gdp_per_capita ($)', 'generation'],
      dtype='object')

In [ ]:
pearson = dat.corr(method='spearman')
pearson

,year,suicides_no,population,suicides/100k pop,HDI for year,gdp_per_capita ($)
year,1.000000,-0.004146,0.014215,-0.029178,0.329094,0.349895
suicides_no,-0.004146,1.000000,0.775580,0.724183,0.184755,0.100735
population,0.014215,0.775580,1.000000,0.215128,0.133076,0.070745
suicides/100k pop,-0.029178,0.724183,0.215128,1.000000,0.152627,0.073063
HDI for year,0.329094,0.184755,0.133076,0.152627,1.000000,0.928608
gdp_per_capita ($),0.349895,0.100735,0.070745,0.073063,0.928608,1.000000
